In [4]:
import pandas as pd

metaanalysis = pd.read_excel("meta.xlsx")
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].astype(str)
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace('.', '')
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace('(', '_')
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace(')', '_')
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace('/', '_')
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace('-', '_')
metaanalysis['Sample_ID'] = metaanalysis['Sample_ID'].str.replace(' ', '_')

metasamples = pd.read_csv("samples.tsv", sep="\t", header=None)
metasamples.columns = ['PATIENT_ID', 'GENDER', "TUMOR", "SAMPLE_ID", "LANE", "FASTQ1", "FASTQ2"]
metasamples = metasamples[metasamples['PATIENT_ID'].isin(metaanalysis['Patient ID']) & metasamples['LANE'] == 1]

In [5]:
with open('command_bam.sh', 'w') as handler:
    for sample in metaanalysis['Sample_ID']:
        handler.write('aws s3 cp s3://scitron/results/Preprocessing/ . --recursive --exclude "*" --include "{}*.recal.bam"\n'.format(sample))
with open('command_vcf.sh', 'w') as handler:
    for index, row in metaanalysis.iterrows():
        sample = row['Sample_ID']
        patient = row['Patient ID']
        has_germline = any(metasamples[metasamples['PATIENT_ID'] == patient]['TUMOR'] == 0)
        cmd = 'Strelka*{}*variants.vcf.gz'.format(sample) if not has_germline else 'StrelkaBP*{}*vs*.vcf.gz'.format(sample)
        handler.write('find /Users/jfnavarro/Projects/Scitron/Results/VariantCalling/ -name "{}" -exec cp \'{{}}\' /Volumes/PowerPuff/ \;\n'.format(cmd))
        if has_germline:
            gdna = metasamples[(metasamples['PATIENT_ID'] == patient)
                   & (metasamples['TUMOR'] == 0)]['SAMPLE_ID'].to_numpy()[0]
            cmd = 'Strelka*{}*variants.vcf.gz'.format(gdna)
            handler.write('find /Users/jfnavarro/Projects/Scitron/Results/VariantCalling/ -name "{}" -exec cp \'{{}}\' /Volumes/PowerPuff/ \;\n'.format(cmd))
            